# Early Stopping Callback
In this assignment you should write a callback that will check the training loss value at the end of each epoch and if it was less than a threshold it will stop training.

In [ ]:
import tensorflow as tf

class EarlyStoppingByLossVal(tf.keras.callbacks.Callback):
    def __init__(self, monitor='loss', value=0.00001, verbose=0):
        super(EarlyStoppingByLossVal, self).__init__()
        #Code Here

## Model Training

In [ ]:
import tensorflow_datasets as tfds
ds, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_ds, validation_ds, test_ds) = ds

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
IMAGE_SIZE = (150, 150)
def preprocess_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

BATCH_SIZE = 32
train_batches = train_ds.shuffle(num_examples // 4).map(preprocess_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_ds.map(preprocess_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_ds.map(preprocess_image).batch(1)

In [ ]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[EarlyStoppingByLossVal(value=.5, verbose=1)])